<a href="https://colab.research.google.com/github/tpmccrea/GradStudentShowcase/blob/master/GradStudentShowcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grad Student Showcase Colab Notebook** 

This notebook contains several sections or cells: 
* Initializing Earth Engine and Loading the Library
* Loading and Displaying Earth Engine Data as Static Images 
* Creating an Interactive Map using the Folium Library

# Initialize EarthEngine

In [2]:
# load GEE library 
import ee

# authenticate EE command line
ee.Authenticate()

# initialize EE command line
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=wMzxQF13FWBzCeMgZgeMHTJSqFDD4ZwqNHXzOmHIkAM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xgF25qlptV87S6NsnYx7ZveEFyhBJI3Z7zw45P3ULBt1fwLN21Kijmg

Successfully saved authorization token.


# Display Static Images of Earth Engine Data

In [3]:
# load libraries
from IPython.display import Image
import ee
# initialize EE command line 
ee.Initialize()
# ee.Authenticate() #execute this command if ee has not been authorized in another cell of the notebook

# import image raster and visualize as a static image, set visual parameters
Elevate = ee.Image('USGS/SRTMGL1_003')
# print the image metadata info using this command 
print(Elevate.getInfo())
Image(url=Elevate.updateMask(Elevate.gt(0))
  .getThumbUrl({'min': 0, 'max': 4500, 'dimensions': 1500,
'palette': ['0099ff', '80ff00', 'fff200', 'ff9d00', 'ff0000']}))

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296001, 417601], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'id': 'USGS/SRTMGL1_003', 'version': 1571719713605057, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'Image', 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="http://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JPL\nat a resolution of 1 arc-second (approximately 30m).</p><p>This dataset has undergone a void-filling process using open-source data\n(ASTER GDEM2, GMTED2010, a

In [4]:
# load libraries
from IPython.display import Image
import ee
# initialize EE command line 
ee.Initialize()
# ee.Authenticate() #execute this command if ee has not been authorized in another cell of the notebook

# import image raster and visualize as a static image, set visual parameters
landforms = ee.Image('CSP/ERGo/1_0/Global/SRTM_landforms')
Image(url=landforms
  .getThumbUrl({'dimensions': 1500,
            'palette': ['141414', '383838' , '808080', 'EBEB8F', 'F7D311', 'AA0000', 'D89382', 'DDC9C9',
                        'DCCDCE', '1C6330', '68AA63', 'B5C98E', 'E1F0E5', 'a975ba', '6f198c'], 'min': 10, 'max': 50}))

# Create Interactive Map

In [6]:
# load libraries
import ee
import folium # folium is library for visualizing data using leaflet maps

# initialize EE command line 
ee.Initialize()
# ee.Authenticate() #execute this command if ee has not been authorized in another cell of the notebook


# define method of operations for displaying GEE data onto map, sets tiles and attributes
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# add drawing method to map, similar to Map.addLayer in the code editor
folium.Map.add_ee_layer = add_ee_layer

# load images, note the differences again between python and javascript api 
OpticalDepth = ee.Image('MODIS/006/MOD08_M3/A2018091')
GlobalTemp = ee.Image('MODIS/006/MOD11A1/2018_09_01')

# set visualization parameters for each of the image layers to be visualized
vis_params = {
  'min': -50,
  'max': 800,
  'palette': ['0500ff', ' 00d4ff', ' 00ff10', ' FFd200', 'FF1e00']}

vis_params1 = {
  'min': 12000,
  'max': 16000,
  'palette': ['0500ff', ' 00d4ff', ' 00ff10', ' FFd200', 'FF1e00']}

# create map object using the folium.Map command 
my_map = folium.Map(location=[20, 0], zoom_start=3, height=1000)

# Add the Optical Depth Image and Global temp Image to Map, set visualization parameters and name layers 
my_map.add_ee_layer(OpticalDepth.select('Aerosol_Optical_Depth_Land_Ocean_Mean_Mean'), vis_params, 'Mean Aerosol Optical Depth')
my_map.add_ee_layer(GlobalTemp.select('LST_Day_1km'), vis_params1, 'Global Surface Temp')

# this adds the layer control panel to the map so that layers can be toggled on and off
my_map.add_child(folium.LayerControl())

# displays the map
display(my_map)

# export map source code as HTML file
my_map.save('index.html')